In [7]:
#EDA
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#to download json_lines: python -m pip install json-lines
#pip install textstat
#pip install --upgrade gensim
from pathlib import Path
import json_lines, json, copy
import numpy as np
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim.downloader as api
import gensim
from nltk.util import ngrams 
import nltk
from nltk.corpus import stopwords
from nltk import download
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from gensim import corpora

import csv
from textstat.textstat import textstat
stop_words = stopwords.words('english')
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.tokenize import TweetTokenizer


#TODO combine lists into csv

#toy example
#rows = zip(list1,list2,list3,list4,list5)
#with open(newfilePath, "w") as f:
#    writer = csv.writer(f)
#    for row in rows:
#        writer.writerow(row)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/EthanHsu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#assorted cleaning
stopWords = stopwords.words('english')

def stem_tokens(tokens, ps):
    stemmed = []
    for item in tokens:
        stemmed.append(ps.stem(item))
    return stemmed
ps = PorterStemmer()

def lemmatize_tokens(tokens, wnl):
    lemmatized = []
    for item in tokens:
        lemmatized.append(wnl.lemmatize(item))
    return lemmatized
wnl = WordNetLemmatizer()


In [9]:
raw_instance_filePath = Path("../clickbait_dataset/instances_train.jsonl")
raw_instance_dir = Path("../clickbait_dataset")
raw_truth_filePath = Path("../clickbait_dataset/truth_train.jsonl")
raw_truth_dir = Path("../clickbait_dataset")

In [10]:
#open with json - raw_instance_train
raw_instance_train=[]
with open('../clickbait_dataset/instances_train.jsonl','rb') as raw_instance_file: #opening file with binary(rb) mode.
    for x in raw_instance_file:
        raw_instance_train.append(json.loads(x))
raw_instance_train[0]

### Fields in instances.jsonl:
##{
#"id" : "<instance id>",
#"postTimestamp" : "<weekday> <month> <day> <hour>:<minute>:<second><time_offset> <year>",
#"postText" : ["<text of the tweet post with links removed>"],
#"postMedia" : ["<path to a file in the media archive>"],
#"targetTitle" : "<title of target news article>",
#"targetDescription": "<description tag of target news article>",
#"targetKeywords" : "<keywords tag of target news article>",
#"targetParagraphs" : ["<text of the i-th paragraph in the target article>"],
#"targetCaptions" : ["<caption of the i-th image in the target article>"]}
### Fields in truth.jsonl:
#{
#"id" : "<instance id>",
#"truthJudgments" : [<number in [0,1]>],
#"truthMean" : <number in [0,1]>,
#"truthMedian" : <number in [0,1]>,
#"truthMode" : <number in [0,1]>,
#truthClass" : "clickbait | no-clickbait"
#}

{'id': '0',
 'postMedia': [],
 'postText': ["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"],
 'postTimestamp': 'Tue Jun 09 16:31:10 +0000 2015',
 'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The Ap

In [11]:
#open with json - raw_truth_train

raw_truth_train=[]
with open('../clickbait_dataset/truth_train.jsonl','rb') as raw_truth_file: #opening file with binary(rb) mode.
    for x in raw_truth_file:
        raw_truth_train.append(json.loads(x))
raw_truth_train[7]


{'id': '7',
 'truthClass': 'no-clickbait',
 'truthJudgments': [0.33333334, 0.0, 0.33333334, 0.33333334, 0.0],
 'truthMean': 0.2,
 'truthMedian': 0.33333334,
 'truthMode': 0.33333334}

In [12]:
#check the properties
print(type(raw_instance_train))    #type for whole data: list
print(type(raw_instance_train[0]))  #type for each element of the liast: dict
print(len(raw_instance_train))
print('Check Dict. Indexes for instance_train: ' + str(raw_instance_train[0].keys()))
print('Check Dict. Indexes for truth_train: ' + str(raw_truth_train[0].keys()))

<class 'list'>
<class 'dict'>
17581
Check Dict. Indexes for instance_train: dict_keys(['targetCaptions', 'targetDescription', 'postText', 'postTimestamp', 'postMedia', 'targetParagraphs', 'targetTitle', 'id', 'targetKeywords'])
Check Dict. Indexes for truth_train: dict_keys(['truthMedian', 'truthMean', 'truthMode', 'truthClass', 'truthJudgments', 'id'])


In [13]:
raw_instance_train = [json.loads(line) for line in open("../clickbait_dataset/instances_train.jsonl")]
raw_truth_train = [json.loads(line) for line in open("../clickbait_dataset/truth_train.jsonl")]

In [14]:
#separate into two list (clickbait/ non_clickbait)
clickbait_train = []
non_clickbait_train = []
for x in range(0, len(raw_instance_train)):
    if raw_truth_train[x]['truthClass'] == 'clickbait':
        temp_dict = copy.deepcopy(raw_instance_train[x])
        temp_dict.update(raw_truth_train[x])
        clickbait_train.append(temp_dict)
    else:
        non_clickbait_train.append(raw_instance_train[x])

In [15]:
#clickbait_train
clickbait_train[0]

{'id': '2',
 'postMedia': [],
 'postText': ['U.S. Soccer should start answering tough questions about Hope Solo, @eric_adelson writes.'],
 'postTimestamp': 'Fri Jun 12 23:36:05 +0000 2015',
 'targetCaptions': ['US to vote for Ali in FIFA election and not Blatter',
  'US to vote for Ali in FIFA election and not Blatter',
  "FILE - This Oct. 10, 2014, file photo shows Sunil Gulati, president of the United States Soccer Federation, during a press conference in Bristol, Conn. The United States says it will vote for Jordan's Prince Ali bin Al-Hussein for FIFA president Friday, May 29, 2015 and not for incumbent Sepp Blatter. (AP Photo/Elise Amendola, File)"],
 'targetDescription': "A U.S. Senator's scathing letter questioned U.S. Soccer's inadequate handling of Solo's domestic violence charges. It's time for Sunil Gulati to respond.",
 'targetKeywords': '',
 'targetParagraphs': ["WINNIPEG, Manitoba – The bubble U.S. Soccer is putting around Hope Solo isn't working to calm anyone's concerns 

In [16]:
#non_clickbait_train
non_clickbait_train[0]

{'id': '0',
 'postMedia': [],
 'postText': ["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"],
 'postTimestamp': 'Tue Jun 09 16:31:10 +0000 2015',
 'targetCaptions': ["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
  "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The Ap

In [17]:
#Post Text tokenizing and word count manually
postTextTokens = []
postTextTokensNoPun = []
postTextCountTokens = []

for x in range(0, len(raw_instance_train)):
    tokens = word_tokenize(raw_instance_train[x]['postText'][0])
    postTextCountTokens.append([x for x in tokens if x != []])
    postTextTokensNoPun.append([c for c in tokens if c not in punctuation])
#print(postTextTokens[0])
#print(len(postTextTokens[0]))

In [18]:
def doc_parser(document):
    #print(document)
    #print(porter_stemmer.stem(document))
    #print(stem_doc)
    tokens = word_tokenize(document)
    non_empty_tokens = [x for x in tokens if x != []]
    clean_tokens = [c for c in non_empty_tokens if c not in punctuation and stop_words]
    #non_apos_tokens = [c.replace("'", "") for c in non_empty_tokens]
    return clean_tokens

In [19]:
def doc_punc_parser(document):
    #print(document)
    #print(porter_stemmer.stem(document))
    #print(stem_doc)
    tokens = word_tokenize(document)
    non_empty_tokens = [x for x in tokens if x != []]
    #clean_tokens = [c for c in non_empty_tokens if c not in punctuation and stop_words]
    #non_apos_tokens = [c.replace("'", "") for c in non_empty_tokens]
    return non_empty_tokens

In [20]:
# remove common words and tokenize
def documentParser(document):  
    #print(punctuation)
    stoplist = set('for a of the and to in'.split())
    tokens = [word for word in document.lower().split() if word not in stoplist and punctuation]
    #print(tokens)
    #stem_tokens = [porter_stemmer.stem(a) for a in tokens]
    #print("stem",stem_tokens)
    non_apos_tokens = [c.replace("'", "").replace(":", "").replace(".", "").replace(",", "") for c in tokens]
    return non_apos_tokens

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
def sklearn_TfidfVectorizer(document):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(document)
    #print(X.shape)
    return vectorizer.get_feature_names()
    

In [22]:
def tweet_Tokenizer(document):
    tknzr = TweetTokenizer()
    #tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
    
    return [c for c in tknzr.tokenize(document) if c not in punctuation]

In [23]:
print(raw_instance_train[0]['postText'])
print("---------------------")
print("postTextCountTokens: ", postTextCountTokens[0], "\n")
print("postTextTokensNoPun: ", postTextTokensNoPun[0], "\n")
print("doc_parser(): ",doc_parser(raw_instance_train[0]['postText'][0]), "\n")
print("doc_punc_parser(): ", doc_punc_parser(raw_instance_train[0]['postText'][0]), "\n")
print("documentParser(): ", documentParser(raw_instance_train[0]['postText'][0]), "\n")
print("sklearn_TfidfVectorizer(): ",sklearn_TfidfVectorizer(raw_instance_train[0]['postText']), "\n")
print("Tweet_Tokenizer(): ", tweet_Tokenizer(raw_instance_train[0]['postText'][0]), "\n")

["Apple's iOS 9 'App thinning' feature will give your phone's storage a boost"]
---------------------
postTextCountTokens:  ['Apple', "'s", 'iOS', '9', "'App", 'thinning', "'", 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost'] 

postTextTokensNoPun:  ['Apple', "'s", 'iOS', '9', "'App", 'thinning', 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost'] 

doc_parser():  ['Apple', "'s", 'iOS', '9', "'App", 'thinning', 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost'] 

doc_punc_parser():  ['Apple', "'s", 'iOS', '9', "'App", 'thinning', "'", 'feature', 'will', 'give', 'your', 'phone', "'s", 'storage', 'a', 'boost'] 

documentParser():  ['apples', 'ios', '9', 'app', 'thinning', 'feature', 'will', 'give', 'your', 'phones', 'storage', 'boost'] 

sklearn_TfidfVectorizer():  ['app', 'apple', 'boost', 'feature', 'give', 'ios', 'phone', 'storage', 'thinning', 'will', 'your'] 

Tweet_Tokenizer():  ["Apple's", 'iOS', '9', 'App', 

In [24]:
print(raw_instance_train[0]['targetTitle'])
documentParser(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


['apple',
 'gives',
 'back',
 'gigabytes',
 'ios',
 '9',
 'app',
 'thinning',
 'feature',
 'will',
 'finally',
 'give',
 'your',
 'phones',
 'storage',
 'boost']

In [25]:
countLongestWord = []

def count_longest_word(word_list):  
    longest_word =  max(word_list , key=len)
    return len(longest_word)

for x in range(0, 1751):
    countLongestWord.append(count_longest_word(postTextTokensNoPun[x]))

print(max(countLongestWord))


26


In [26]:
print(postTextTokensNoPun[915])
count_longest_word(postTextTokensNoPun[915])

['Fury', 'as', 'men', 'throw', 'a', 'cat', 'into', 'a', 'crocodile-infested', 'pond']


18

In [27]:
#Lexicon (Word) Count
targetTitleCountWords = []
for x in range(0, len(raw_instance_train)):
    targetTitleCountWords.append(textstat.lexicon_count(raw_instance_train[x]['targetTitle'][0], removepunct=True))

In [28]:
#Grade Level reading difficulty
#Coleman-Liau index, Gunning FOG Formula, Linsear Write Formula
targetTitle_coleman_liau_index = []
targetTitle_gunning_fog = []
targetTitle_linsear_write_formula = []
for x in range(0, len(raw_instance_train)):
    targetTitle_coleman_liau_index.append(textstat.coleman_liau_index(raw_instance_train[x]['targetTitle']))
    targetTitle_gunning_fog.append(textstat.gunning_fog(raw_instance_train[x]['targetTitle']))
    targetTitle_linsear_write_formula.append(textstat.linsear_write_formula(raw_instance_train[x]['targetTitle']))
targetTitle_coleman_liau_index = []


In [29]:
print(raw_instance_train[0]['targetTitle'])
textstat.coleman_liau_index(raw_instance_train[0]['targetTitle'])


Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


12.12

In [30]:
print(raw_instance_train[0]['targetTitle'])
textstat.gunning_fog(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


20.564705882352943

In [31]:
print(raw_instance_train[0]['targetTitle'])
textstat.linsear_write_formula(raw_instance_train[0]['targetTitle'])

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


6.5

In [55]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)
word_vectors = api.load("glove-wiki-gigaword-100")

In [56]:
def avg_sim(title, body):
    title = api(title)
    return np.mean(map(lambda x: title.similarity(api(x)), body))

In [65]:
#calculating similarity between titles and body.
import spacy
#nlp = spacy.load('../GoogleNews-vectors-negative300.bin')

def avg_doc_sim(nlp, title, body):
#    similarity = word_vectors.n_similarity(title, body)
#    return "{:.4f}".format(similarity)
    print(title)
    title = nlp(title)
    print(title)
    
    return np.mean(map(lambda x: title.similarity(nlp(x)), body))

In [73]:
nlp_en = spacy.load('en_core_web_md')
doc1 = nlp_en(u"'Obama speaks to the media in Illinois'")
doc2 = nlp_en(u"The president greets the press in Chicago")
doc1.similarity(doc2)

0.80762849634064104

In [78]:
#use this one
def spacy_acg_sim(title, body):
    body_list = []
    title = nlp_en(title)
    for b in body:
        body_list.append(title.similarity(nlp_en(b)))
    return np.mean(body_list)

In [79]:
np.mean([0, 0.1, 0.8, 0.9])

0.45000000000000001

In [74]:
sentence_obama = 'Obama speaks to the media in Illinois'#.lower().split()
sentence_president = 'The president greets the press in Chicago'#.lower().split()

In [76]:
title_doc1 = nlp_en(raw_instance_train[0]['targetTitle'])
paragraph_doc2 = nlp_en(raw_instance_train[0]['targetParagraphs'][0])
title_doc1.similarity(paragraph_doc2)

0.85945738934493399

In [81]:
#avg_similairty_result
spacy_acg_sim(clickbait_train[0]['targetTitle'],clickbait_train[0]['targetParagraphs'])

0.71853533655838242

In [82]:
train_list = []
for x in range(0, 10):
    train_list.append(spacy_acg_sim(clickbait_train[x]['targetTitle'],clickbait_train[x]['targetParagraphs']))
train_list

[0.71853533655838242,
 0.69679372245086335,
 0.81921675845200193,
 0.73781977982365066,
 0.76393140957036754,
 0.56885461412387084,
 0.79082285528907958,
 0.61477369948484828,
 0.60736286439950149,
 0.8295308684770043]

In [68]:
#checking glove-wiki-gigaword-100 and ../GoogleNews-vectors-negative300.bin performance
#print("glove-wiki-gigaword-100:",avg_doc_sim(word_vectors,sentence_obama,sentence_president))
print("GoogleNews:", avg_doc_sim(model,raw_instance_train[0]['targetTitle'], raw_instance_train[0]['targetParagraphs']))
#raw_instance_train

Apple gives back gigabytes: iOS 9 'app thinning' feature will finally give your phone's storage a boost


TypeError: 'Word2VecKeyedVectors' object is not callable

In [588]:
print("glove-wiki-gigaword-100:", avg_doc_sim(word_vectors,['sushi', 'shop'], ['japanese', 'restaurant']))
print("GoogleNews:", avg_doc_sim(model,['sushi', 'shop'], ['japanese', 'restaurant']))

glove-wiki-gigaword-100: 0.7067
GoogleNews: 0.5984


In [804]:
import re

def cleanhtml_(raw_html):
    cleanr = re.compile('<.*?>')
    print(raw_html.replace(u'\xa0',u''))
    return raw_html.replace(u'\xa0',u'')
    cleantext_ = re.sub(cleanr, '', clean_html)
    #return (cleantext_)
    return cleantext_

In [805]:
str1 = "lives.\xa0"

In [806]:
cleanhtml_(str1)

lives.


'lives.'

In [815]:
#Parsing targetTitle and targetParagraphs
#print([raw_instance_train[0]['targetTitle']])
c_target_title_ = tweet_Tokenizer(raw_instance_train[0]['targetTitle'])
c_target_paragraph_ = []
for n in raw_instance_train[0]['targetParagraphs']:
    n = n.replace(u'\xa0',u'').replace(':', ' ')
    if(n != ""):
        #n = [n]
        c_target_paragraph_ += tweet_Tokenizer(n)

In [816]:
#review parsing result for targetTitle and targetParagraphs
print(c_target_title_) #target title
print("------")
print(c_target_paragraph_) # target paragraph

['Apple', 'gives', 'back', 'gigabytes', 'iOS', '9', 'app', 'thinning', 'feature', 'will', 'finally', 'give', 'your', "phone's", 'storage', 'a', 'boost']
------
['Paying', 'for', 'a', '64GB', 'phone', 'only', 'to', 'discover', 'that', 'this', 'is', 'significantly', 'reduced', 'by', 'system', 'files', 'and', 'bloatware', 'is', 'the', 'bane', 'of', 'many', 'smartphone', "owner's", 'lives', 'And', 'the', 'issue', 'became', 'so', 'serious', 'earlier', 'this', 'year', 'that', 'some', 'Apple', 'users', 'even', 'sued', 'the', 'company', 'over', 'it', 'But', 'with', 'the', 'launch', 'of', 'iOS', '9', 'Apple', 'is', 'hoping', 'to', 'address', 'storage', 'concerns', 'by', 'introducing', 'a', 'feature', 'known', 'as', 'app', 'thinning', 'It', 'has', 'been', 'explained', 'on', 'the', 'watchOS', 'Developer', 'Library', 'site', 'and', 'is', 'aimed', 'at', 'developers', 'looking', 'to', 'optimise', 'their', 'apps', 'to', 'work', 'on', 'iOS', 'and', 'the', 'watchOS', 'It', 'ensures', 'apps', 'use', 'th

In [817]:
#measure title and body's similarity
#error "word 'watchos' not in vocabulary"
avg_doc_sim(model,c_target_title_, c_target_paragraph_)

KeyError: "word 'phone's' not in vocabulary"

In [337]:
#targetCaptions
targetCaptions_documents = raw_instance_train[0]['targetCaptions']
targetCaptions_documents

["'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
 "'App thinning' will be supported on Apple's iOS 9 and later models. It ensures apps use the lowest amount of storage space on a device by only downloading the parts it needs to run on individual handsets. It 'slices' the app into 'app variants' that only need to access the specific files on that specific device",
 "The guidelines also discuss so-called 'on-demand resources.' This allows developers to omit features from an app until they are opened or requested by the user. The App Store hosts these resources on Apple servers and manages the downloads for the developer and user.\xa0This will also increase how quickly an app downloads",
 "The guidelines also discuss so-calle

In [336]:
# find common words and tokenize
def pre_word_count(documents):
    stoplist = set('for a of the and to in'.split())
    texts = [[word for word in document.lower().split() if word not in stoplist]
             for document in documents]
    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [[token for token in text if frequency[token] > 1]
             for text in texts]
    return texts

In [210]:
#count each word in a document
def word_Count(parsed_document):
    dictionary = corpora.Dictionary(parsed_document)
    return dictionary.token2id

In [338]:
#finding common words
targetCaptions_parsed_document = pre_word_count(targetCaptions_documents)
targetCaptions_parsed_document

[["'app",
  "thinning'",
  'will',
  'be',
  'supported',
  'on',
  "apple's",
  'ios',
  '9',
  'later',
  'models.',
  'it',
  'ensures',
  'apps',
  'use',
  'lowest',
  'amount',
  'storage',
  'space',
  'on',
  'device',
  'by',
  'only',
  'downloading',
  'parts',
  'it',
  'needs',
  'run',
  'on',
  'individual',
  'handsets.',
  'it',
  "'slices'",
  'app',
  'into',
  "'app",
  "variants'",
  'that',
  'only',
  'need',
  'access',
  'specific',
  'files',
  'on',
  'that',
  'specific',
  'device'],
 ["'app",
  "thinning'",
  'will',
  'be',
  'supported',
  'on',
  "apple's",
  'ios',
  '9',
  'later',
  'models.',
  'it',
  'ensures',
  'apps',
  'use',
  'lowest',
  'amount',
  'storage',
  'space',
  'on',
  'device',
  'by',
  'only',
  'downloading',
  'parts',
  'it',
  'needs',
  'run',
  'on',
  'individual',
  'handsets.',
  'it',
  "'slices'",
  'app',
  'into',
  "'app",
  "variants'",
  'that',
  'only',
  'need',
  'access',
  'specific',
  'files',
  'on',
 

In [339]:
#word count
word_Count(targetCaptions_parsed_document)

{'%': 83,
 "'app": 0,
 "'concealing,": 115,
 "'on-demand": 37,
 "'purge": 69,
 "'slices'": 1,
 "(apple's": 70,
 '(illustrated)': 84,
 '16gb': 116,
 '20%': 117,
 '54%': 85,
 '56gb': 86,
 '5c': 87,
 '6': 88,
 '64gb': 89,
 '79%': 90,
 '87.5': 91,
 '8gb,': 92,
 '9': 2,
 'access': 3,
 'accusing': 118,
 'advertised': 119,
 'after': 93,
 'allows': 38,
 'also': 39,
 'amount': 4,
 'an': 40,
 'app': 5,
 'apple': 41,
 "apple's": 6,
 'apps': 7,
 'apps,': 94,
 'are': 42,
 'available': 120,
 'available.': 121,
 'be': 8,
 'between': 95,
 'by': 9,
 'capacity': 122,
 'developer': 43,
 'developers': 44,
 'device': 10,
 "disclose'": 123,
 'discuss': 45,
 'disgruntled': 124,
 'disk': 71,
 'downloading': 11,
 'downloads': 46,
 'drop': 96,
 'earlier': 125,
 'ensures': 12,
 'failing': 126,
 'features': 47,
 'filed': 127,
 'files': 13,
 'free': 97,
 'free.': 98,
 'from': 48,
 'galaxy': 99,
 'giant': 128,
 'graph': 129,
 'guidelines': 49,
 'handsets,': 100,
 'handsets.': 14,
 'hosts': 50,
 'how': 51,
 'include

In [ ]:
#pos_tag

In [204]:
text = word_tokenize("And now for something completely different")
nltk.pos_tag(text)

[('And', 'CC'),
 ('now', 'RB'),
 ('for', 'IN'),
 ('something', 'NN'),
 ('completely', 'RB'),
 ('different', 'JJ')]

In [399]:
#clickbait_train, emoji
def pos_1gram(data, title):
    postag_words = word_tokenize(data.get(title)[0])
    return nltk.pos_tag(postag_words)

pos_tag_list = []
#for single_data in clickbait_train:
    #print(pos_2gram(single_data, "postText"))

In [401]:
pos_1gram(clickbait_train[0], "postText")

[('U.S.', 'NNP'),
 ('Soccer', 'NNP'),
 ('should', 'MD'),
 ('start', 'VB'),
 ('answering', 'VBG'),
 ('tough', 'JJ'),
 ('questions', 'NNS'),
 ('about', 'IN'),
 ('Hope', 'NNP'),
 ('Solo', 'NNP'),
 (',', ','),
 ('@', 'NNP'),
 ('eric_adelson', 'NN'),
 ('writes', 'NNS'),
 ('.', '.')]

In [566]:
#calculate tags in a data
def cal_pos_1grams_tag(data, title, tag):
    i = 0
    pos_list = []
    postag_words = word_tokenize(data.get(title)[0])
    pos_list = nltk.pos_tag(postag_words)
    for tags in pos_list:
        print("tags: ", tags)
        if tags[1] == tag:
            i+=1
    return i   

In [565]:
cal_pos_1grams_tag(clickbait_train[0], "postText", "NNP")

tags:  ('U.S.', 'NNP')
tags:  ('Soccer', 'NNP')
tags:  ('should', 'MD')
tags:  ('start', 'VB')
tags:  ('answering', 'VBG')
tags:  ('tough', 'JJ')
tags:  ('questions', 'NNS')
tags:  ('about', 'IN')
tags:  ('Hope', 'NNP')
tags:  ('Solo', 'NNP')
tags:  (',', ',')
tags:  ('@', 'NNP')
tags:  ('eric_adelson', 'NN')
tags:  ('writes', 'NNS')
tags:  ('.', '.')


5

In [561]:
#yutarochan's code
def pos_2gram(title, p1, p2):
    pos_list = pos_tag(word_tokenize(title))
    #print("-1: ", pos_list[:-1])
    #print("1: ", pos_list[1:])
    #print("zip: ", list(zip(pos_list[:-1], pos_list[1:])))
    #print(list(zip(pos_list[:-1], pos_list[1:]))[0][1])
    #print(list(zip(pos_list[:-1], pos_list[1:]))[1][1])
    return sum(map(lambda x: x[0][1] == p1 and x[1][1] == p2, zip(pos_list[:-1], pos_list[1:])))

In [562]:
clickbait_train[0]['targetTitle']

'U.S. Soccer should start answering tough questions about Hope Solo'

In [563]:
pos_2gram(clickbait_train[0]['targetTitle'], 'NNP', 'NNP')

2

In [470]:
#revised pos_2grams
def pos_2gram_sum(title, p1, p2):
    pos_2g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2g_num += 1
    return pos_2g_num

In [471]:
pos_2gram_sum(clickbait_train[0]['targetTitle'], 'NNP', 'NNP')

2

In [472]:
#check pos_2grams
def pos_2gram_check(title, p1, p2):
    pos_2gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-1], pos_list[1:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2:
            pos_2gram_list.append([pos_z[0], pos_z[1]])
    return pos_2gram_list

In [473]:
print(pos_2gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP'))

[[('U.S.', 'NNP'), ('Soccer', 'NNP')], [('Hope', 'NNP'), ('Solo', 'NNP')]]


In [484]:
#revised pos_3grams
def pos_3gram_sum(title, p1, p2, p3):
    pos_3g_num = 0
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3g_num += 1
    return pos_3g_num

In [493]:
print(pos_3gram_sum(clickbait_train[0]['targetTitle'], 'NNP', 'NNP', 'NNP'))

0


In [494]:
#check pos_3grams
def pos_3gram_check(title, p1, p2, p3):
    pos_3gram_list = []
    pos_list = pos_tag(word_tokenize(title))
    pos_ziplist = zip(pos_list[:-2], pos_list[1:-1], pos_list[2:])
    for pos_z in pos_ziplist:
        if pos_z[0][1] == p1 and pos_z[1][1] == p2 and pos_z[2][1] == p3:
            pos_3gram_list.append([pos_z[0], pos_z[1],pos_z[2]])
    return pos_3gram_list

In [495]:
print(pos_3gram_check(clickbait_train[0]['targetTitle'], 'NNP', 'NNP', 'NNP'))

[]


In [507]:
#just downloading some nltk samples
nltk.download('brown')
nltk.corpus.brown.tagged_words(categories='news')

[nltk_data] Downloading package brown to /Users/EthanHsu/nltk_data...
[nltk_data]   Package brown is already up-to-date!


[('The', 'AT'), ('Fulton', 'NP-TL'), ...]

In [508]:
#just downloading some nltk samples
nltk_list = list(nltk.corpus.brown.tagged_words(categories='news'))

In [552]:
target_para_list = []
for n in clickbait_train[0]['targetParagraphs']:
    target_para_ = doc_punc_parser(n)
    #print(type(target_para_list))
    #print(target_para_)
    target_para_list = target_para_list + target_para_
    #print(n)
    #print(nltk.pos_tag(n))

target_para_tags = nltk.pos_tag(target_para_list)

In [557]:
def count_tags (tags_list):
    tag_fd = nltk.FreqDist(tag for (word, tag) in tags_list)
    return (tag_fd.most_common())

In [558]:
print(count_tags(target_para_tags))

[('NN', 115), ('NNP', 90), ('DT', 88), ('IN', 87), ('JJ', 44), ('VBZ', 41), ('RB', 40), ('.', 31), ('VB', 30), (',', 27), ('NNS', 24), ('TO', 23), ('CC', 20), ('POS', 16), ('VBD', 15), ('VBG', 14), ('PRP', 13), ('VBN', 12), ('PRP$', 10), ("''", 10), ('``', 9), ('VBP', 6), ('MD', 6), ('JJR', 5), ('WP', 4), ('WRB', 3), (':', 3), ('CD', 3), ('EX', 3), ('JJS', 2), ('(', 2), ('WDT', 2), (')', 2), ('RBR', 2), ('RBS', 1), ('RP', 1), ('NNPS', 1)]


In [497]:
def findtags(tag_prefix, tagged_text):
    cfd = nltk.ConditionalFreqDist((tag, word) for (word, tag) in tagged_text
                                  if tag.startswith(tag_prefix))
    return dict((tag, cfd[tag].most_common(5)) for tag in cfd.conditions())

In [560]:
findtags("NN",target_para_tags)

{'NN': [('violence', 6),
  ('lot', 4),
  ('incident', 3),
  ('explanation', 3),
  ('game', 3)],
 'NNP': [('Solo', 15),
  ('U.S.', 11),
  ('Soccer', 10),
  ('Gulati', 5),
  ('Hope', 4)],
 'NNPS': [('Americans', 1)],
 'NNS': [('women', 2),
  ('questions', 2),
  ('months', 2),
  ('concerns', 1),
  ('values', 1)]}